In [1]:
import numpy as np
import astropy
from astropy.table import Table, Column,join,vstack
from astropy import units as u
from astropy.coordinates import SkyCoord, search_around_sky
from IPython.display import clear_output
import pickle
import os
from glob import glob
from shutil import copyfile
import pymoc
from pymoc import MOC
from pymoc.util import catalog
from pymoc.io.fits import read_moc_fits
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.neighbors import KernelDensity

import importlib
import time

In [2]:

#importlib.reload(mltier)
from mltier import (get_center, get_n_m, estimate_q_m, Field, SingleMLEstimator, MultiMLEstimator,
                     parallel_process, get_sigma_all, get_q_m, get_threshold, q0_min_level, q0_min_numbers,
                     get_n_m_kde, estimate_q_m_kde, get_q_m_kde, describe)
from Q0_calc import Q0_calc

In [3]:
def varstat(distribution):
    """
    Print basic properties about a variable to stdout


    Parameters:
    ------------

    distribution : The input variable/distribution to display the statistic of to stdout

    Returns:
    -----------
    """

    stat_to_print = [np.nanmean(distribution),np.nanmedian(distribution),
                     np.nanstd(distribution),len(distribution),np.nanmin(distribution),
                     np.nanmax(distribution),len(distribution[distribution==0.])]

    var_to_print = ["Mean", "Median", "Std. Dev.", "Length", "Min", "Max", "Len_Zeros"]
    var_to_print = [aa.ljust(10) for aa in var_to_print]
    print(" ".join(var_to_print))

    #stat_to_print = map(str,stat_to_print)
    #print(stat_to_print)
    stat_to_print = [str(np.round(bb,6)).ljust(10) for bb in stat_to_print]
    stat_to_print[4] = str(np.nanmin(distribution))
    print(" ".join(stat_to_print))
    # print(stat_to_print)
    return

#### Read in the optical and infrared data and the radio data

In [5]:
opt_ir_cat = Table.read('../../../HELP/dmu_products/dmu32/dmu32_Lockman-SWIRE/data/Lockman-SWIRE_20180219.fits')

In [6]:
print(len(opt_ir_cat))
opt_ir_cat[0]

4366298


help_id,field,ra,dec,hp_idx,f_ap_wfc_u,ferr_ap_wfc_u,m_ap_wfc_u,merr_ap_wfc_u,f_wfc_u,ferr_wfc_u,m_wfc_u,merr_wfc_u,f_ap_wfc_g,ferr_ap_wfc_g,m_ap_wfc_g,merr_ap_wfc_g,f_wfc_g,ferr_wfc_g,m_wfc_g,merr_wfc_g,f_ap_wfc_r,ferr_ap_wfc_r,m_ap_wfc_r,merr_ap_wfc_r,f_wfc_r,ferr_wfc_r,m_wfc_r,merr_wfc_r,f_ap_wfc_i,ferr_ap_wfc_i,m_ap_wfc_i,merr_ap_wfc_i,f_wfc_i,ferr_wfc_i,m_wfc_i,merr_wfc_i,f_ap_wfc_z,ferr_ap_wfc_z,m_ap_wfc_z,merr_ap_wfc_z,f_wfc_z,ferr_wfc_z,m_wfc_z,merr_wfc_z,f_ap_gpc1_g,ferr_ap_gpc1_g,m_ap_gpc1_g,merr_ap_gpc1_g,f_gpc1_g,ferr_gpc1_g,m_gpc1_g,merr_gpc1_g,f_ap_gpc1_r,ferr_ap_gpc1_r,m_ap_gpc1_r,merr_ap_gpc1_r,f_gpc1_r,ferr_gpc1_r,m_gpc1_r,merr_gpc1_r,f_ap_gpc1_i,ferr_ap_gpc1_i,m_ap_gpc1_i,merr_ap_gpc1_i,f_gpc1_i,ferr_gpc1_i,m_gpc1_i,merr_gpc1_i,f_ap_gpc1_z,ferr_ap_gpc1_z,m_ap_gpc1_z,merr_ap_gpc1_z,f_gpc1_z,ferr_gpc1_z,m_gpc1_z,merr_gpc1_z,f_ap_gpc1_y,ferr_ap_gpc1_y,m_ap_gpc1_y,merr_ap_gpc1_y,f_gpc1_y,ferr_gpc1_y,m_gpc1_y,merr_gpc1_y,f_ap_megacam_u,ferr_ap_megacam_u,m_ap_megacam_u,merr_ap_megacam_u,f_megacam_u,ferr_megacam_u,m_megacam_u,merr_megacam_u,f_ap_megacam_g,ferr_ap_megacam_g,m_ap_megacam_g,merr_ap_megacam_g,f_megacam_g,ferr_megacam_g,m_megacam_g,merr_megacam_g,f_ap_megacam_r,ferr_ap_megacam_r,m_ap_megacam_r,merr_ap_megacam_r,f_megacam_r,ferr_megacam_r,m_megacam_r,merr_megacam_r,f_ap_megacam_z,ferr_ap_megacam_z,m_ap_megacam_z,merr_ap_megacam_z,f_megacam_z,ferr_megacam_z,m_megacam_z,merr_megacam_z,f_ap_ukidss_k,ferr_ap_ukidss_k,m_ap_ukidss_k,merr_ap_ukidss_k,f_ukidss_k,ferr_ukidss_k,m_ukidss_k,merr_ukidss_k,f_ap_irac_i3,ferr_ap_irac_i3,m_ap_irac_i3,merr_ap_irac_i3,f_irac_i3,ferr_irac_i3,m_irac_i3,merr_irac_i3,f_ap_irac_i4,ferr_ap_irac_i4,m_ap_irac_i4,merr_ap_irac_i4,f_irac_i4,ferr_irac_i4,m_irac_i4,merr_irac_i4,f_ap_irac_i1,ferr_ap_irac_i1,m_ap_irac_i1,merr_ap_irac_i1,f_irac_i1,ferr_irac_i1,m_irac_i1,merr_irac_i1,f_ap_irac_i2,ferr_ap_irac_i2,m_ap_irac_i2,merr_ap_irac_i2,f_irac_i2,ferr_irac_i2,m_irac_i2,merr_irac_i2,f_ap_ukidss_j,ferr_ap_ukidss_j,m_ap_ukidss_j,merr_ap_ukidss_j,f_ukidss_j,ferr_ukidss_j,m_ukidss_j,merr_ukidss_j,m_megacam_i,ferr_megacam_i,merr_megacam_i,f_megacam_i,m_megacam_y,ferr_megacam_y,merr_megacam_y,f_megacam_y,stellarity,stellarity_origin,zspec,zspec_qual,zspec_association_flag,ebv,f_mips_24,ferr_mips_24,flag_mips_24,f_pacs_green,ferr_pacs_green,flag_pacs_green,f_pacs_red,ferr_pacs_red,flag_pacs_red,f_spire_250,ferr_spire_250,flag_spire_250,f_spire_350,ferr_spire_350,flag_spire_350,f_spire_500,ferr_spire_500,flag_spire_500,redshift,flag_megacam_i,flag_megacam_y,flag_cleaned,flag_merged,flag_gaia,flag_optnir_obs,flag_optnir_det,flag_gpc1_g,flag_gpc1_r,flag_gpc1_i,flag_gpc1_z,flag_gpc1_y,flag_irac_i1,flag_irac_i2,flag_irac_i3,flag_irac_i4,flag_wfc_u,flag_megacam_u,flag_wfc_g,flag_megacam_g,flag_wfc_r,flag_megacam_r,flag_wfc_i,flag_wfc_z,flag_megacam_z,cigale_mstar,cigale_mstar_err,cigale_sfr,cigale_sfr_err,cigale_dustlumin,cigale_dustlumin_err,cigale_dustlumin_ironly,cigale_dustlumin_ironly_err,flag_cigale,flag_cigale_opt,flag_cigale_ir,flag_cigale_ironly
,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,mag,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uJy,uJy,,,uJy,uJy,,,uJy,uJy,,,uJy,uJy,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,uJy,,,mJy,,,mJy,,,mJy,,,mJy,,,mJy,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bytes33,bytes18,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float

In [7]:
lofar_all = Table.read('../lofar/deep_fields/Lockman/data/edited_cats/radio/image_full_ampphase_di_m.NS_shift.blanked.scaled.cat_foverlap.fits')

In [8]:
lofar_all[0]

Source_id,Isl_id,RA,E_RA,DEC,E_DEC,Total_flux,E_Total_flux,Peak_flux,E_Peak_flux,RA_max,E_RA_max,DEC_max,E_DEC_max,Maj,E_Maj,Min,E_Min,PA,E_PA,Maj_img_plane,E_Maj_img_plane,Min_img_plane,E_Min_img_plane,PA_img_plane,E_PA_img_plane,DC_Maj,E_DC_Maj,DC_Min,E_DC_Min,DC_PA,E_DC_PA,DC_Maj_img_plane,E_DC_Maj_img_plane,DC_Min_img_plane,E_DC_Min_img_plane,DC_PA_img_plane,E_DC_PA_img_plane,Isl_Total_flux,E_Isl_Total_flux,Isl_rms,Isl_mean,Resid_Isl_rms,Resid_Isl_mean,S_Code,FLAG_OVERLAP,flag_clean,Source_Name
,,deg,deg,deg,deg,Jy,Jy,Jy / beam,Jy / beam,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,Jy,Jy,Jy / beam,Jy / beam,Jy / beam,Jy / beam,,,,
int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,int64,int64,bytes22
0,2,167.140371012,0.000224831119071,57.7293926059,6.98686836164e-05,0.000392924930152,0.000121253904068,0.000378014226818,6.43930593582e-05,167.140371012,0.000224831119071,57.7293926059,6.98686836164e-05,0.00231057449509,0.000530889673946,0.00125120866868,0.000158018298889,97.1852631296,19.0337910704,0.00230769444596,0.000530889673946,0.00125118242094,0.000158018298889,92.6292703714,19.0337910704,0.0,0.000530889673946,0.0,0.000158018298889,0.0,19.0337910704,0.0,0.000530889673946,0.0,0.000158018298889,0.0,19.0337910704,0.000351682817313,7.53038720014e-05,6.83596881572e-05,0.0,2.9045888823e-06,-4.4299389157e-10,S,0,1,ILTJ110833.69+574345.8


In [9]:
opt_colname = 'm_ap_megacam_r'
ir_colname = 'm_ap_irac_i2'
opt_ra = 'ra'
opt_dec = 'dec'

#### Define the field 

In [10]:
#select the moc of the optical data
#MOC_loc = '../lofar/deep_fields/Lockman/data/Bootes_I-band_MOC.fits'
#MHz150_MOC = MOC(filename=MOC_loc)

# Load in the MOCs
PATH_MOC_r = "../lofar/deep_fields/Lockman/data/edited_cats/optical/mocs/LH_r_moc_order_18_MOC_with_add.fits"
PATH_MOC_se2 = "../lofar/deep_fields/Lockman/data/edited_cats/optical/mocs/LH_se2_moc_order_18_MOC.fits"
PATH_MOC_sw2 = "../lofar/deep_fields/Lockman/data/edited_cats/optical/mocs/LH_sw2_moc_order_18_MOC.fits"

moc_r = pymoc.MOC()
read_moc_fits(moc_r, PATH_MOC_r)

moc_se2 = pymoc.MOC()
read_moc_fits(moc_se2, PATH_MOC_se2)

moc_sw2 = pymoc.MOC()
read_moc_fits(moc_sw2, PATH_MOC_sw2)

print("Area of r-MOC: {0} sq. deg.".format(moc_r.area_sq_deg))
print("Area of se2-MOC: {0} sq. deg.".format(moc_se2.area_sq_deg))
print("Area of sw2-MOC: {0} sq. deg.".format(moc_sw2.area_sq_deg))

moc_overlap = moc_r.intersection(moc_sw2)

print("Area of overlaping moc: {0} sq. deg.".format(moc_overlap.area_sq_deg))

Area of r-MOC: 13.321917803932173 sq. deg.
Area of se2-MOC: 5.574102257180664 sq. deg.
Area of sw2-MOC: 10.949807996199757 sq. deg.
Area of overlaping moc: 10.73406473187727 sq. deg.


In [11]:
ra_down, ra_up, dec_down, dec_up = min(lofar_all['RA']),max(lofar_all['RA']),min(lofar_all['DEC']),max(lofar_all['DEC'])
field = Field(ra_down, ra_up, dec_down, dec_up, moc_overlap)

field area is defined by a MOC


#### Filter the catalogues

In [12]:
# remove sources with a major axis greater than 30 arcseconds
lofar_aux = lofar_all[~np.isnan(lofar_all['Maj'])]
lofar = field.filter_catalogue(lofar_aux[(lofar_aux['Maj'] < 10./3600) & (lofar_aux['S_Code']=='S')], 
                               colnames=("RA", "DEC"))
print('number of radio sources is: {}'.format(len(lofar)))

filtering in a MOC
number of radio sources is: 21620


In [13]:
opt_ir = field.filter_catalogue(opt_ir_cat, 
                               colnames=(opt_ra, opt_dec))

#remove sources with no detection in the opt or ir bands used for xmatching
#mask = (~np.isnan(opt_ir[opt_colname])) | (~np.isnan(opt_ir[ir_colname]))
#mask = opt_ir['FLAG_DEEP']==1 
#opt_ir = opt_ir[mask]

filtering in a MOC


In [14]:
opt_ir["colour"] = opt_ir[opt_colname] - opt_ir[ir_colname]

In [15]:
opt_ir_index = np.arange(len(opt_ir))

In [16]:
opt_ir_coords = SkyCoord(opt_ir[opt_ra], 
                           opt_ir[opt_dec], 
                           unit=(u.deg, u.deg), 
                           frame='icrs')

In [17]:
lofar_coords = SkyCoord(lofar['RA'], 
                       lofar['DEC'], 
                       unit=(u.deg, u.deg), 
                       frame='icrs')

In [18]:
#remove sources with mag=-99
opt_ir_num_both = (~np.isnan(opt_ir[opt_colname]) & ~np.isnan(opt_ir[ir_colname])) # number of sources detected in opt and ir
opt_ir_num_opt = (~np.isnan(opt_ir[opt_colname]) & np.isnan(opt_ir[ir_colname])) # number of sources detected in opt only
opt_ir_num_ir =(np.isnan(opt_ir[opt_colname]) & ~np.isnan(opt_ir[ir_colname])) # number of sources detected in ir only
opt_num = (opt_ir_num_both | opt_ir_num_opt) & (opt_ir[opt_colname]!=-99)
ir_num = (opt_ir_num_both | opt_ir_num_ir) & (opt_ir[ir_colname]!=-99)

opt_cat = opt_ir[opt_num]
ir_cat = opt_ir[ir_num]

In [19]:
print("Total    - ", len(opt_ir))
print("opt and ir - ", np.sum(opt_ir_num_both))
print("Only opt   - ", np.sum(opt_ir_num_opt))
print("With opt   - ", np.sum(opt_num))
print("Only ir  - ", np.sum(opt_ir_num_ir))
print("With ir  - ", np.sum(ir_num))

Total    -  2830987
opt and ir -  561799
Only opt   -  1319555
With opt   -  1881354
Only ir  -  385409
With ir  -  947208


check the distribution of sources to make sure the selected area is correct and that there are no gaps that could influence the calculation of Q0

In [ ]:
x = lofar['RA']
y = lofar['DEC']
plt.scatter(x,y,s=0.1)

x = opt_ir[opt_ra]
y = opt_ir[opt_dec]
plt.scatter(x,y,s=0.01)
plt.show()

In [ ]:
mask = ~np.isnan(opt_ir_cat['m_gpc1_i'])
x = opt_ir_cat['ra'][mask]
y = opt_ir_cat['dec'][mask]
plt.scatter(x,y,s=0.01)
plt.xlim(158,159)
plt.ylim(58,59)
plt.show()

In [ ]:
print(min(lofar['RA']),max(lofar['RA']),min(lofar['DEC']),max(lofar['DEC']))

### Compute Q0 for the optical and IR

In [20]:
Q0_opt,search_rad_r = Q0_calc(lofar,opt_cat,'RA','DEC',opt_ra,opt_dec,opt_colname,min(lofar['RA']),max(lofar['RA']),min(lofar['DEC']),max(lofar['DEC']),0.1,8,30,moc=moc_overlap)
#Q0_opt,search_rad_r = Q0_calc(lofar,opt_cat,'RA','DEC',opt_ra,opt_dec,opt_colname,16,163.5,57.5,58.5,0.1,8,30,moc=moc_overlap)

field area is defined by a MOC
filtering in a MOC
filtering in a MOC
[    0     1     2 ..., 21617 21618 21619]
13816
starting to find Q0. This will take a while
finding Q0 with radius = 0.1 arcseconds
ratio of the random generation area compared to the moc area is: 3.350025161472081
number of random sources with no matches: 13810
number of real sources with no matches: 13533
ratio of the random generation area compared to the moc area is: 3.350025161472081
number of random sources with no matches: 13810
number of real sources with no matches: 13533
ratio of the random generation area compared to the moc area is: 3.350025161472081
number of random sources with no matches: 13813
number of real sources with no matches: 13533
ratio of the random generation area compared to the moc area is: 3.350025161472081
number of random sources with no matches: 13809
number of real sources with no matches: 13533
ratio of the random generation area compared to the moc area is: 3.350025161472081
number 

KeyboardInterrupt: 

In [ ]:
x = search_rad_r
y = 1 - np.array(Q0_opt)
plt.plot(x,y)
plt.xlabel('Radius (arcseconds)')
plt.ylabel('Q0')
plt.show()

Q0 = max(Q0_opt)
#search_rad_opt = search_rad_r[Q0_opt == Q0][0]
#using large search rad to make sure every source has a crossmatch and to make sure no true counterparts are missed
search_rad_opt = 15
#Q0 = popt[0]
#search_rad = 6.9103

print('Q0 = {}'.format(Q0))
print('search radius = {}'.format(search_rad_opt))
Q0_opt = Q0


In [ ]:
#onlt run this cell if you have Q0 calculated previosly
search_rad_opt = 15
Q0_opt = 0.758524446898

In [ ]:
Q0_ir,search_rad_r = Q0_calc(lofar,ir_cat,'RA','DEC',opt_ra,opt_dec,ir_colname,min(lofar['RA']),max(lofar['RA']),min(lofar['DEC']),max(lofar['DEC']),0.1,8,30,moc=moc_overlap)

In [ ]:
x = search_rad_r
y = 1 - np.array(Q0_ir)
plt.plot(x,y)
plt.xlabel('Radius (arcseconds)')
plt.ylabel('Q0')
plt.show()
Q0 = max(Q0_ir)

#search_rad_ir = search_rad_r[Q0_ir == Q0][0]
search_rad_ir = 15
#Q0 = popt[0]
#search_rad = 6.9103

print('Q0 = {}'.format(Q0))
print('search radius = {}'.format(search_rad_ir))
Q0_ir = Q0

### Prepare the optical likelihood ratio

In [ ]:
opt_cat = opt_ir[opt_num]

In [ ]:
bin_list_opt = np.arange(7,35, 0.05)
center_opt = get_center(bin_list_opt)

In [ ]:
bandwidth_opt = 0.5

#find the magnitude distribution of all sources n(m)
n_m_opt = get_n_m(opt_cat[opt_colname], bin_list_opt, field.area)
#find the magnitude distribution of all sources using a kernal instead of bining 
n_m_opt_kde = get_n_m_kde(opt_cat[opt_colname], center_opt, field.area, bandwidth=bandwidth_opt)
n_m_opt_kde_cs = np.cumsum(n_m_opt_kde)

#estimate q(m) using the method of Fleuren et al 2012
q_m_opt = estimate_q_m(opt_cat[opt_colname], 
                      bin_list_opt, 
                      n_m_opt, 
                      lofar_coords, 
                      opt_ir_coords[opt_num], 
                      radius=5)

#estimate q(m) using the method of fleuren et al 2012 but instead of binning the magnitudes using a
#kernal to create a smoother q(m)
q_m_opt_kde = estimate_q_m_kde(opt_cat[opt_colname], 
                      center_opt, 
                      n_m_opt_kde, 
                      lofar_coords, 
                      opt_ir_coords[opt_num], 
                      radius=5, 
                      bandwidth=bandwidth_opt)
q_m_opt_kde_cs = np.cumsum(q_m_opt_kde)

In [ ]:
print(len(opt_cat[opt_colname]))
print(field.area/3600/3600)

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)
plt.subplot(1,3,1)
plt.plot(center_opt, n_m_opt,color='blue');
plt.plot(center_opt, n_m_opt_kde_cs,color='red');
plt.title('n (m)')


plt.subplot(1,3,2)
plt.rcParams["figure.figsize"] = (5,5)
plt.plot(center_opt, q_m_opt,color='blue');
plt.plot(center_opt, q_m_opt_kde_cs,color='red');
plt.title('q (m)')

plt.subplot(1,3,3)
plt.rcParams["figure.figsize"] = (5,5)
plt.plot(center_opt, q_m_opt/n_m_opt,color='blue');
plt.plot(center_opt, q_m_opt_kde_cs/n_m_opt_kde_cs,color='red');
plt.ylim(0.9*np.min(q_m_opt_kde_cs/n_m_opt_kde_cs),1.1*np.max(q_m_opt_kde_cs/n_m_opt_kde_cs))
plt.title('q (m) / n (m)')
plt.show()

print('maximum value of n(m) is: {}'.format(np.max(n_m_opt_kde_cs)))
varstat(n_m_opt_kde_cs)

In [ ]:
'''save = [n_m_opt_kde_cs,q_m_opt_kde_cs]
pickle.dump(save,open('help_opt_nqm.pkl','wb'))'''

In [ ]:
varstat(q_m_opt_kde_cs)

In [ ]:
likelihood_ratio_opt = SingleMLEstimator(Q0_opt, n_m_opt_kde_cs, q_m_opt_kde_cs, center_opt)

### Prepare the IR likelihood ratio

In [ ]:
ir_cat = opt_ir[ir_num]

In [ ]:
bin_list_ir = np.arange(7, 35, 0.05)
center_ir = get_center(bin_list_ir)

In [ ]:
bandwidth_ir = 0.5

mask = ~np.isnan(ir_cat[ir_colname])
n_m_ir = get_n_m(ir_cat[ir_colname][mask], bin_list_ir, field.area)
n_m_ir_kde = get_n_m_kde(ir_cat[ir_colname][mask], center_ir, field.area, bandwidth=bandwidth_ir)
n_m_ir_kde_cs = np.cumsum(n_m_ir_kde)

q_m_ir = estimate_q_m(ir_cat[ir_colname][mask], 
                      bin_list_ir, 
                      n_m_ir, 
                      lofar_coords, 
                      opt_ir_coords[ir_num][mask], 
                      radius=5)

q_m_ir_kde = estimate_q_m_kde(ir_cat[ir_colname][mask], 
                      center_ir, 
                      n_m_ir_kde, 
                      lofar_coords, 
                      opt_ir_coords[ir_num][mask], 
                      radius=5, 
                      bandwidth=bandwidth_ir)
q_m_ir_kde_cs = np.cumsum(q_m_ir_kde)

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
plt.subplot(1,3,1)
plt.plot(center_ir, n_m_ir,color='blue');
plt.plot(center_ir, n_m_ir_kde_cs,color='red');
plt.title('n (m)')

plt.subplot(1,3,2)
plt.rcParams["figure.figsize"] = (5,5)
plt.plot(center_ir, q_m_ir,color='blue');
plt.plot(center_ir, q_m_ir_kde_cs,color='red');
plt.title('q (m)')

plt.subplot(1,3,3)
plt.rcParams["figure.figsize"] = (5,5)
plt.plot(center_ir, q_m_ir/n_m_ir,color='blue');
plt.plot(center_ir, q_m_ir_kde_cs/n_m_ir_kde_cs,color='red');
plt.ylim(0.9*np.min(q_m_ir_kde_cs/n_m_ir_kde_cs),1.1*np.max(q_m_ir_kde_cs/n_m_ir_kde_cs))
plt.title('q (m) / n (m)')
plt.show()

In [ ]:
save = [n_m_ir_kde_cs,q_m_ir_kde_cs]
pickle.dump(save,open('help_ir_nqm.pkl','wb'))

In [ ]:
likelihood_ratio_ir = SingleMLEstimator(Q0_ir, n_m_ir_kde_cs, q_m_ir_kde_cs, center_ir)

### Start the magnitude only crossmatch

#### optical

In [ ]:
import multiprocessing
n_cpus_total = multiprocessing.cpu_count()
n_cpus = max(1, n_cpus_total-1)

In [ ]:
radius = 15
idx_lofar, idx_opt, d2d, d3d = search_around_sky(
    lofar_coords, opt_ir_coords[opt_num], radius*u.arcsec)

# create an array of the indexs of lofar sources with a possible crossmatch within the search radius
idx_lofar_unique = np.unique(idx_lofar)
print('number of LOFAR sources with a possible crossmatch within {} arcseconds is {}'.format(radius,len(idx_lofar_unique)))

In [ ]:
lofar["lr_opt"] = np.nan                   # Likelihood ratio
lofar["lr_dist_opt"] = np.nan              # Distance to the selected source
lofar["lr_index_opt"] = np.nan             # Index of the PanSTARRS source in combined
lofar["lr_reliability_opt"] = np.nan
lofar["helpid_opt"] = ''

In [ ]:
#do the crossmatching for the radio to optical dataset
#inputs: i, the index saying which radio source to do the LR analysis for
def ml_opt(i):

    #get the indexes to the radio and optical catalogues from the cross match done above
    idx_0 = idx_opt[idx_lofar == i]
    d2d_0 = d2d[idx_lofar == i]
    mag = opt_cat[opt_colname][idx_0]
    
    lofar_ra = lofar[i]["RA"]
    lofar_dec = lofar[i]["DEC"]
    lofar_pa = lofar[i]["PA"]
    lofar_maj_err = lofar[i]["E_Maj"]
    lofar_min_err = lofar[i]["E_Min"]
    c_ra = opt_cat[opt_ra][idx_0]
    c_dec = opt_cat[opt_dec][idx_0]
    c_ra_err = 0.35
    c_dec_err = 0.35
    
    #calculate the positional error of the radio source based on the PA and major and minor axis
    #also includes an additional error of 0.6 arcseconds added in quadrature 
    sigma, sigma_maj, sigma_min = get_sigma_all(lofar_maj_err, lofar_min_err, lofar_pa, 
                      lofar_ra, lofar_dec, 
                      c_ra, c_dec, c_ra_err, c_dec_err)
    
    lr_0 = likelihood_ratio_opt(mag, d2d_0.arcsec, sigma)
    #if the radio source has no sources within the search radius then return a row of nans
    if len(lr_0) == 0:
        result = [np.nan,np.nan,np.nan,np.nan,np.nan]
        return result
    chosen_index = np.argmax(lr_0)
    lr_sum = np.sum(lr_0)
    #calculate the realiability using the formula from smith et al 2013
    rel = lr_0/(lr_sum + (1-Q0_opt))
    helpid = opt_cat['ID'][idx_0][0]
    #return the indexs to optical catalogue, the distance to the source, the likelihood ratio
    #and the helpid
    result = [opt_ir_index[opt_num][idx_0[chosen_index]], # Index
              (d2d_0.arcsec)[chosen_index],                        # distance
              lr_0[chosen_index],
              rel[chosen_index],
              helpid]                                  # LR
    

    return(result)
    #return (result,rel[chosen_index],helpid)

In [ ]:
res = mltier.parallel_process(idx_lofar_unique, ml_opt, n_jobs=n_cpus)

In [ ]:
#add the crossmatched sources to the radio table
(lofar["lr_index_opt"][idx_lofar_unique], 
 lofar["lr_dist_opt"][idx_lofar_unique], 
 lofar["lr_opt"][idx_lofar_unique],
 lofar["lr_reliability_opt"][idx_lofar_unique],
 lofar["helpid_opt"][idx_lofar_unique]) = list(map(list, zip(*res)))

In [ ]:
lofar["lr_opt"][np.isnan(lofar["lr_opt"])] = 0

In [ ]:
#calculate the threshold of trusted crossmatches using Q0 as the fraction of sources that have trusted crossmatches
mask = (~np.isnan(lofar['lr_dist_opt'])) & (lofar['lr_opt']>0)
threshold_opt = np.percentile(lofar["lr_opt"][mask], 100*(1 - Q0_opt))
print('the LR thereshold is: {}'.format(threshold_opt))
varstat(lofar['lr_opt'])

In [ ]:
plt.rcParams["figure.figsize"] = (15,6)
plt.subplot(1,2,1)
plt.hist(lofar[lofar["lr_opt"] != 0]["lr_opt"], bins=200)
plt.vlines([threshold_opt], 0, 1000)
plt.ylim([0,1000])

plt.subplot(1,2,2)
plt.hist(np.log10(lofar[lofar["lr_opt"] != 0]["lr_opt"]+1), bins=200)
plt.vlines(np.log10(threshold_opt+1), 0, 1000)
ticks, _ = plt.xticks()
plt.xticks(ticks, ["{:.1f}".format(10**t-1) for t in ticks])
plt.ylim([0,1000]);
plt.show()

In [ ]:
lofar["lr_index_sel_opt"] = lofar["lr_index_opt"]
lofar["lr_index_sel_opt"][lofar["lr_opt"] < threshold_opt] = np.nan
print('number of xmatches above the threshold is {}'.format(np.sum(lofar['lr_opt']>threshold_opt)))

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
bins = np.arange(0,search_rad_opt,search_rad_opt/50)
plt.subplot(1,3,1)
mask = ~np.isnan(lofar['lr_dist_opt'])
plt.hist(lofar['lr_dist_opt'][mask],bins=bins,color='red')
mask = (lofar['lr_opt'] > threshold_opt) & (~np.isnan(lofar['lr_dist_opt']))
plt.hist(lofar['lr_dist_opt'][mask],bins=bins,color='green')
plt.xlabel('Seperation (arcseconds)')
green_patch = mpatches.Patch(color='green',label='Trusted Crossmatches')
red_patch = mpatches.Patch(color='red',label='All Crossmatches')
plt.legend(handles=[red_patch,green_patch])

plt.subplot(1,3,2)
reliability_r = []
mask1 = (~np.isnan(lofar['lr_dist_opt']))
for n in range(len(bins)-1):
    mask = (lofar['lr_dist_opt'] > bins[n]) & (lofar['lr_dist_opt'] < bins[n+1])
    mask = mask
    rel_temp = lofar['lr_reliability_opt'][mask]
    reliability_r.append(np.mean(rel_temp))
rad_cent = get_center(bins)
plt.plot(rad_cent,reliability_r,color='green')
plt.xlabel('Seperation (arcseconds)')
plt.ylabel('Reliability')

plt.subplot(1,3,3)
lrt_r = []
mask1 = (~np.isnan(lofar['lr_dist_opt']))
for n in range(len(bins)-1):
    mask = (lofar['lr_dist_opt'] > bins[n]) & (lofar['lr_dist_opt'] < bins[n+1])
    #mask = mask[mask1]
    rel_temp = lofar['lr_opt'][mask]
    lrt_r.append(np.mean(rel_temp))
rad_cent = get_center(bins)
plt.plot(rad_cent,lrt_r,color='green')
plt.xlabel('Seperation (arcseconds)')
plt.ylabel('likelihood ratio')
plt.ylim(0,np.max(lrt_r)*1.1)

plt.show()

#### IR band

In [ ]:
idx_lofar, idx_ir, d2d, d3d = search_around_sky(
    lofar_coords, opt_ir_coords[ir_num], radius*u.arcsec)

idx_lofar_unique = np.unique(idx_lofar)
print('number of LOFAR sources with a possible crossmatch within {} arcseconds is {}'.format(radius,len(idx_lofar_unique)))

In [ ]:
len(opt_ir_coords[ir_num])

In [ ]:
max(idx_ir)

In [ ]:
radius = 15
idx_lofar, idx_ir, d2d, d3d = search_around_sky(
    lofar_coords, opt_ir_coords[ir_num], radius*u.arcsec)

# create an array of the indexs of lofar sources with a possible crossmatch within the search radius
idx_lofar_unique = np.unique(idx_lofar)
print('number of LOFAR sources with a possible crossmatch within {} arcseconds is {}'.format(radius,len(idx_lofar_unique)))

In [ ]:
lofar["lr_ir"] = np.nan                   # Likelihood ratio
lofar["lr_dist_ir"] = np.nan              # Distance to the selected source
lofar["lr_index_ir"] = np.nan             # Index of the PanSTARRS source in combined
lofar["lr_reliability_ir"] = np.nan
lofar["helpid_ir"] = ''

In [ ]:
def ml_ir(i):

    idx_0 = idx_ir[idx_lofar == i]
    d2d_0 = d2d[idx_lofar == i]
    mag = ir_cat[ir_colname][idx_0]
    
    lofar_ra = lofar[i]["RA"]
    lofar_dec = lofar[i]["DEC"]
    lofar_pa = lofar[i]["PA"]
    lofar_maj_err = lofar[i]["E_Maj"]
    lofar_min_err = lofar[i]["E_Min"]
    c_ra = ir_cat[opt_ra][idx_0]
    c_dec = ir_cat[opt_dec][idx_0]
    c_ra_err = 0.35
    c_dec_err = 0.35
    
    sigma, sigma_maj, sigma_min = get_sigma_all(lofar_maj_err, lofar_min_err, lofar_pa, 
                      lofar_ra, lofar_dec, 
                      c_ra, c_dec, c_ra_err, c_dec_err)
    
    lr_0 = likelihood_ratio_ir(mag, d2d_0.arcsec, sigma)
    if len(lr_0) == 0:
        result = [np.nan,np.nan,np.nan,np.nan,np.nan]
        return result
    chosen_index = np.argmax(lr_0)
    lr_sum = np.sum(lr_0)
    rel = lr_0/(lr_sum + (1-Q0_opt))
    helpid = ir_cat['ID'][idx_0][0]
    result = [opt_ir_index[ir_num][idx_0[chosen_index]], # Index
              (d2d_0.arcsec)[chosen_index],                        # distance
              lr_0[chosen_index],
              rel[chosen_index],
              helpid]                                  # LR
    

    return(result)
    #return (result,rel[chosen_index],helpid)

In [ ]:
res = parallel_process(idx_lofar_unique, ml_ir, n_jobs=n_cpus)

In [ ]:
(lofar["lr_index_ir"][idx_lofar_unique], 
 lofar["lr_dist_ir"][idx_lofar_unique], 
 lofar["lr_ir"][idx_lofar_unique],
 lofar["lr_reliability_ir"][idx_lofar_unique],
 lofar["helpid_ir"][idx_lofar_unique]) = list(map(list, zip(*res)))

In [ ]:
lofar["lr_ir"][np.isnan(lofar["lr_ir"])] = 0

In [ ]:
mask = (~np.isnan(lofar['lr_dist_ir'])) & (lofar['lr_ir']>0)
threshold_ir = np.percentile(lofar["lr_ir"][mask], 100*(1 - Q0_ir))
threshold_ir

In [ ]:
np.sum(lofar['lr_ir']<threshold_ir)

In [ ]:
plt.rcParams["figure.figsize"] = (15,6)
plt.subplot(1,2,1)
plt.hist(lofar[lofar["lr_ir"] != 0]["lr_ir"], bins=200)
plt.vlines([threshold_ir], 0, 1000)
plt.ylim([0,1000])

plt.subplot(1,2,2)
plt.hist(np.log10(lofar[lofar["lr_ir"] != 0]["lr_ir"]+1), bins=200)
plt.vlines(np.log10(threshold_ir+1), 0, 1000)
ticks, _ = plt.xticks()
plt.xticks(ticks, ["{:.1f}".format(10**t-1) for t in ticks])
plt.ylim([0,1000]);
plt.show()

In [ ]:
lofar["lr_index_sel_ir"] = lofar["lr_index_ir"]
lofar["lr_index_sel_ir"][lofar["lr_ir"] < threshold_ir] = np.nan
print('number of xmatches above the threshold is {}'.format(np.sum(lofar['lr_opt']>threshold_ir)))

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
bins = np.arange(0,search_rad_opt,search_rad_opt/50)
plt.subplot(1,3,1)
mask = ~np.isnan(lofar['lr_dist_ir'])
plt.hist(lofar['lr_dist_ir'][mask],bins=bins,color='red')
mask = (lofar['lr_ir'] > threshold_ir) & (~np.isnan(lofar['lr_dist_ir']))
plt.hist(lofar['lr_dist_ir'][mask],bins=bins,color='green')
plt.xlabel('Seperation (arcseconds)')
green_patch = mpatches.Patch(color='green',label='Trusted Crossmatches')
red_patch = mpatches.Patch(color='red',label='All Crossmatches')
plt.legend(handles=[red_patch,green_patch])

plt.subplot(1,3,2)
reliability_r = []
mask1 = (~np.isnan(lofar['lr_dist_ir']))
for n in range(len(bins)-1):
    mask = (lofar['lr_dist_ir'] > bins[n]) & (lofar['lr_dist_ir'] < bins[n+1])
    mask = mask
    rel_temp = lofar['lr_reliability_ir'][mask]
    reliability_r.append(np.mean(rel_temp))
rad_cent = get_center(bins)
plt.plot(rad_cent,reliability_r,color='green')
plt.xlabel('Seperation (arcseconds)')
plt.ylabel('Reliability')

plt.subplot(1,3,3)
lrt_r = []
mask1 = (~np.isnan(lofar['lr_dist_ir']))
for n in range(len(bins)-1):
    mask = (lofar['lr_dist_ir'] > bins[n]) & (lofar['lr_dist_ir'] < bins[n+1])
    #mask = mask[mask1]
    rel_temp = lofar['lr_ir'][mask]
    lrt_r.append(np.mean(rel_temp))
rad_cent = get_center(bins)
plt.plot(rad_cent,lrt_r,color='green')
plt.xlabel('Seperation (arcseconds)')
plt.ylabel('likelihood ratio')
plt.ylim(0,np.max(lrt_r)*1.1)

plt.show()

### Final selection of the match

We combine the ML matching done in i-band and W1-band. All the galaxies were the LR is above the selection ratio for the respective band are finally selected.

In [ ]:
# lr_opt_and_ir = (lofar["lr_opt"] != 0) & (lofar["lr_ir"] != 0)
# lr_only_opt = (lofar["lr_opt"] != 0) & (lofar["lr_ir"] == 0)
# lr_only_ir = (lofar["lr_opt"] == 0) & (lofar["lr_ir"] != 0)
# lr_no_match = (lofar["lr_opt"] == 0) & (lofar["lr_ir"] == 0)
lr_opt_and_ir = ~np.isnan(lofar["lr_index_sel_opt"]) & ~np.isnan(lofar["lr_index_sel_ir"])
lr_only_opt = ~np.isnan(lofar["lr_index_sel_opt"]) & np.isnan(lofar["lr_index_sel_ir"])
lr_only_ir = np.isnan(lofar["lr_index_sel_opt"]) & ~np.isnan(lofar["lr_index_sel_ir"])
lr_no_match = np.isnan(lofar["lr_index_sel_opt"]) & np.isnan(lofar["lr_index_sel_ir"])

In [ ]:
print(np.sum(lr_opt_and_ir))
print(np.sum(lr_only_opt))
print(np.sum(lr_only_ir))
print(np.sum(lr_no_match))

In [ ]:
lofar["lr_index_1"] = np.nan
lofar["lr_dist_1"] = np.nan
lofar["lr_1"] = np.nan
lofar["lr_type_1"] = 0

In [ ]:
# Only i matches
lofar["lr_1"][lr_only_opt] = lofar["lr_opt"][lr_only_opt]
lofar["lr_index_1"][lr_only_opt] = lofar["lr_index_opt"][lr_only_opt]
lofar["lr_dist_1"][lr_only_opt] = lofar["lr_dist_opt"][lr_only_opt]
lofar["lr_type_1"][lr_only_opt] = 1

# Only w1 matches
lofar["lr_1"][lr_only_ir] = lofar["lr_ir"][lr_only_ir]
lofar["lr_index_1"][lr_only_ir] = lofar["lr_index_ir"][lr_only_ir]
lofar["lr_dist_1"][lr_only_ir] = lofar["lr_dist_ir"][lr_only_ir]
lofar["lr_type_1"][lr_only_ir] = 2

# Both matches
lofar["lr_1"][lr_opt_and_ir] = np.max([lofar["lr_opt"][lr_opt_and_ir], lofar["lr_ir"][lr_opt_and_ir]], axis=0)
lofar["lr_type_1"][lr_opt_and_ir] = np.argmax([lofar["lr_opt"][lr_opt_and_ir], lofar["lr_ir"][lr_opt_and_ir]], axis=0) + 1

c1 = (lofar["lr_type_1"] == 1)
c2 = (lofar["lr_type_1"] == 2)
lofar["lr_index_1"][lr_opt_and_ir & c1] = lofar["lr_index_opt"][lr_opt_and_ir & c1]
lofar["lr_index_1"][lr_opt_and_ir & c2] = lofar["lr_index_ir"][lr_opt_and_ir & c2]
lofar["lr_dist_1"][lr_opt_and_ir & c1] = lofar["lr_dist_opt"][lr_opt_and_ir & c1]
lofar["lr_dist_1"][lr_opt_and_ir & c2] = lofar["lr_dist_ir"][lr_opt_and_ir & c2]

In [ ]:
print("match    sel-opt: ", np.sum(lofar["lr_type_1"][lr_opt_and_ir] == 1))
print("match   sel-ir: ", np.sum(lofar["lr_type_1"][lr_opt_and_ir] == 2))
print("match     both: ", np.sum(lofar["lr_type_1"][lr_opt_and_ir] == 1) + 
                          np.sum(lofar["lr_type_1"][lr_opt_and_ir] == 2))
print("match   opt-only: ", np.sum(lofar["lr_type_1"] == 1) - np.sum(lofar["lr_type_1"][lr_opt_and_ir] == 1))
print("match  ir-only: ", np.sum(lofar["lr_type_1"] == 2) - np.sum(lofar["lr_type_1"][lr_opt_and_ir] == 2))
print("match      all: ", np.sum(lofar["lr_type_1"] == 1) + 
                          np.sum(lofar["lr_type_1"] == 2))
print("         Total: ", len(lofar))
print('percentage of sources with a crossmatch is: {}'.format((np.sum(lofar["lr_type_1"] == 1) +np.sum(lofar["lr_type_1"] == 2))/len(lofar)))

In [ ]:
print('number of sources for which the match in opt-band and ir-band are above the threshold but gives a different match to the combined catalogue is {}'.format(np.sum(lofar["lr_index_opt"][lr_opt_and_ir] != lofar["lr_index_ir"][lr_opt_and_ir])))

### Define the colour bins to be used later

Create color bins in the opt-ir plane.

In [ ]:
bins=np.arange(-12,10,0.25)
y = opt_ir['colour']
plt.hist(y,bins=bins,log=True)
plt.show()

In [ ]:
colour_limits = [0.0, 0.5, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.5, 4.0]
colour_limits = np.arange(-5,5,0.25)

In [ ]:
# start with the only opt or ir detected and the colour < lowest colour bin 
colour_bin_def = [{"name":"only W1", "condition": opt_ir_num_ir},
                  {"name":"only i", "condition": opt_ir_num_opt},
                  {"name":"-inf to {}".format(colour_limits[0]), 
                   "condition": (opt_ir["colour"] < colour_limits[0])}]

# Get the colour bins
for i in range(len(colour_limits)-1):
    name = "{} to {}".format(colour_limits[i], colour_limits[i+1])
    condition = ((opt_ir["colour"] >= colour_limits[i]) & 
                 (opt_ir["colour"] < colour_limits[i+1]))
    colour_bin_def.append({"name":name, "condition":condition})

# Add the "more than higher colour" bin
colour_bin_def.append({"name":"{} to inf".format(colour_limits[-1]), 
                       "condition": (opt_ir["colour"] >= colour_limits[-1])})

In [ ]:
# assign every source a number based on which colour bin it is in
opt_ir["category"] = np.nan
for i in range(len(colour_bin_def)):
    opt_ir["category"][colour_bin_def[i]["condition"]] = i

In [ ]:
numbers_combined_bins = np.array([np.sum(a["condition"]) for a in colour_bin_def])
numbers_combined_bins

## Skipped bit on duplicated sources

In [ ]:
bin_list = [bin_list_ir if i == 0 else bin_list_opt for i in range(len(colour_bin_def))]
centers = [center_ir if i == 0 else center_opt for i in range(len(colour_bin_def))]

numbers_combined_bins = np.array([np.sum(a["condition"]) for a in colour_bin_def])
bandwidth_colour = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
numbers_combined_bins

In [ ]:
#Calculate n(m) and q(m) within each colour bin to give n(m,c) and q(m,c)
n_m_kde = []

# W1 only sources
n_m_kde.append(get_n_m_kde(opt_ir[ir_colname][opt_ir["category"] == 0], 
                       centers[0], field.area, bandwidth=bandwidth_colour[0]))

# Rest of the sources
for i in range(1, len(colour_bin_def)):
    n_m_kde.append(get_n_m_kde(opt_ir[opt_colname][opt_ir["category"] == i], 
                           centers[i], field.area, bandwidth=bandwidth_colour[i]))

In [ ]:
n_m = []

# W1 only sources
n_m.append(get_n_m(opt_ir[ir_colname][opt_ir["category"] == 0], bin_list_ir, field.area))

# Rest of the sources
for i in range(1, len(colour_bin_def)):
    n_m.append(get_n_m(opt_ir[opt_colname][opt_ir["category"] == i], bin_list_opt, field.area))

In [ ]:
plt.rcParams["figure.figsize"] = (15,15)
for i, n_m_k in enumerate(n_m_kde):
    plt.subplot(5,5,i+1)
    plt.plot(centers[i], n_m[i])
    plt.plot(centers[i], np.cumsum(n_m_k))
plt.show()

In [ ]:
lofar["category"] = np.nan
lofar[ir_colname] = np.nan
lofar[opt_colname] = np.nan

In [ ]:
c = ~np.isnan(lofar["lr_index_1"])
indices = lofar["lr_index_1"][c].astype(int)
lofar["category"][c] = opt_ir[indices]["category"]
lofar[ir_colname][c] = opt_ir[indices][ir_colname]
lofar[opt_colname][c] = opt_ir[indices][opt_colname]

The next parameter represent the number of matched LOFAR sources in each colour category.

In [ ]:
numbers_lofar_combined_bins = np.array([np.sum(lofar["category"] == c) 
                                        for c in range(len(numbers_combined_bins))])
numbers_lofar_combined_bins

In [ ]:
#Q0(c) is defined as the fraction of trsuted crossmatches in each category to give a value of Q0 in each category
#and Q0(c) is the sum of these individual Q0's divided by the total number of radio sources
Q_0_colour = numbers_lofar_combined_bins/len(lofar)
q0_total = np.sum(Q_0_colour)
q0_total

In [ ]:
#calculate q(m) in each colour bin to calculate q(m,c)
q_m = []
radius = 15. 

# W1 only sources
q_m.append(get_q_m(lofar[ir_colname][lofar["category"] == 0], 
                   bin_list_ir, 
                   numbers_lofar_combined_bins[0], 
                   n_m[0], 
                   field.area, 
                   radius=radius))

# Rest of the sources
for i in range(1, len(numbers_lofar_combined_bins)):
    q_m.append(get_q_m(lofar[opt_colname][lofar["category"] == i], 
                   bin_list_opt, 
                   numbers_lofar_combined_bins[i], 
                   n_m[i], 
                   field.area, 
                   radius=radius))

In [ ]:
q_m_kde = []
radius = 15. 

# W1 only sources
q_m_kde.append(get_q_m_kde(lofar[ir_colname][lofar["category"] == 0], 
                   centers[0], 
                   radius=radius,
                   bandwidth=0.5))

# Rest of the sources
for i in range(1, len(numbers_lofar_combined_bins)):
    q_m_kde.append(get_q_m_kde(lofar[opt_colname][lofar["category"] == i], 
                   centers[i], 
                   radius=radius,
                   bandwidth=bandwidth_colour[i]))

In [ ]:
np.ones_like(23)
centers[0]

In [ ]:
np.ones_like(centers[0])*0.5

In [ ]:
plt.rcParams["figure.figsize"] = (15,15)
for i, q_m_k in enumerate(q_m_kde):
    plt.subplot(5,5,i+1)
    plt.plot(centers[i], q_m[i])
    plt.plot(centers[i], np.cumsum(q_m_k))
plt.show()

In [ ]:
#plot q(m,c)/n(m,c) to see that everthing looks okay. The width of the line shows how many sources are in the bin
plt.rcParams["figure.figsize"] = (12,10)

from matplotlib import cm
from matplotlib.collections import LineCollection

cm_subsection = np.linspace(0., 1., 16) 
colors = [ cm.viridis(x) for x in cm_subsection ]

low = np.nonzero(centers[1] >= 15)[0][0]
high = np.nonzero(centers[1] >= 22.2)[0][0]

fig, a = plt.subplots()

for i, q_m_k in enumerate(q_m):
    #plot(centers[i], q_m_old[i]/n_m_old[i])
    a = plt.subplot(4,4,i+1)
    if i not in [-1]:
        n_m_aux = n_m[i]/np.sum(n_m[i])
        lwidths = (n_m_aux/np.max(n_m_aux)*10).astype(float) + 1
        #print(lwidths)
        
        y_aux = q_m_k/n_m[i]
        mask = ~np.isnan(y_aux)
        y_aux = y_aux[mask]
        factor = np.max(y_aux[low:high])
        y = y_aux
        #print(y)
        x = centers[i]
        
        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        
        lc = LineCollection(segments, linewidths=lwidths, color=colors[i])
        
        a.add_collection(lc)
        
        #plot(centers[i], x/factor, color=colors[i-1])
        plt.xlim([12, 30])
        if i == 0:
            plt.xlim([10, 23])
        plt.ylim([0, 1.2*factor])

plt.subplots_adjust(left=0.125, 
                bottom=0.1, 
                right=0.9, 
                top=0.9,
                wspace=0.4, 
                hspace=0.2)
plt.show()

In [ ]:
selection = ~np.isnan(opt_ir["category"]) # Avoid the two dreaded sources with no actual data
catalogue = opt_ir[selection]

In [ ]:
radius = 15

In [ ]:
def apply_ml(i, likelihood_ratio_function):
    idx_0 = idx_i[idx_lofar == i]
    d2d_0 = d2d[idx_lofar == i]
    
    category = catalogue["category"][idx_0].astype(int)
    mag = catalogue[opt_colname][idx_0]
    mag[category == 0] = catalogue[ir_colname][idx_0][category == 0]
    
    lofar_ra = lofar[i]["RA"]
    lofar_dec = lofar[i]["DEC"]
    lofar_pa = lofar[i]["PA"]
    lofar_maj_err = lofar[i]["E_Maj"]
    lofar_min_err = lofar[i]["E_Min"]
    c_ra = catalogue["ra"][idx_0]
    c_dec = catalogue["dec"][idx_0]
    c_ra_err = 1.0
    c_dec_err = 1.0
    
    sigma, sigma_maj, sigma_min = get_sigma_all(lofar_maj_err, lofar_min_err, lofar_pa, 
                      lofar_ra, lofar_dec, 
                      c_ra, c_dec, c_ra_err, c_dec_err)

    lr_0 = likelihood_ratio_function(mag, d2d_0.arcsec, sigma, category)
    if len(lr_0) == 0:

        result = [np.nan,np.nan,np.nan,np.nan,np.nan]
        return result
    chosen_index = np.argmax(lr_0)
    lr_sum = np.sum(lr_0)
    rel = lr_0/(lr_sum + (1-q0_total))
    helpid = catalogue['help_id'][idx_0][0]
    result = [opt_ir_index[selection][idx_0[chosen_index]], # Index
              (d2d_0.arcsec)[chosen_index],                        # distance
              lr_0[chosen_index],
              rel[chosen_index],
              helpid]                                  # LR
    return result

### Run the cross-match

This will not need to be repeated after

In [ ]:
idx_lofar, idx_i, d2d, d3d = search_around_sky(
    lofar_coords, opt_ir_coords[selection], radius*u.arcsec)
idx_lofar_unique = np.unique(idx_lofar)

In [ ]:
likelihood_ratio = MultiMLEstimator(Q_0_colour, n_m_kde, q_m_kde, centers)

In [ ]:
def ml(i):
    return apply_ml(i, likelihood_ratio)

In [ ]:
res = parallel_process(idx_lofar_unique, ml, n_jobs=n_cpus)

In [ ]:
lofar["lr_index_2"] = np.nan
lofar["lr_dist_2"] = np.nan
lofar["lr_2"] = np.nan
lofar["lr_reliability_2"] = np.nan
lofar["helpid_2"] = ''

In [ ]:
(lofar["lr_index_2"][idx_lofar_unique], 
 lofar["lr_dist_2"][idx_lofar_unique], 
 lofar["lr_2"][idx_lofar_unique],
 lofar["lr_reliability_2"][idx_lofar_unique],
 lofar["helpid_2"][idx_lofar_unique]) = list(map(list, zip(*res)))

In [ ]:
lofar["lr_2"][np.isnan(lofar["lr_2"])] = 0

mask = (~np.isnan(lofar["lr_dist_2"])) & (lofar['lr_2']>0)
threshold = np.percentile(lofar["lr_2"][mask], 100*(1 - q0_total))
threshold

In [ ]:
plt.rcParams["figure.figsize"] = (15,6)
plt.subplot(1,2,1)
plt.hist(lofar[lofar["lr_2"] != 0]["lr_2"], bins=200)
plt.vlines([threshold], 0, 1000)
plt.ylim([0,1000])
plt.subplot(1,2,2)
plt.hist(np.log10(lofar[lofar["lr_2"] != 0]["lr_2"]+1), bins=200)
plt.vlines(np.log10(threshold+1), 0, 1000)
ticks, _ = plt.xticks()
plt.xticks(ticks, ["{:.1f}".format(10**t-1) for t in ticks])
plt.ylim([0,1000]);
plt.show()

In [ ]:
lofar["lr_index_sel_2"] = lofar["lr_index_2"]
lofar["lr_index_sel_2"][lofar["lr_2"] < threshold] = np.nan

In [ ]:
n_changes = np.sum((lofar["lr_index_sel_2"] != lofar["lr_index_1"]) & 
                   ~np.isnan(lofar["lr_index_sel_2"]) &
                   ~np.isnan(lofar["lr_index_1"]))
n_changes

In [ ]:
lofar["category"] = np.nan
lofar[ir_colname] = np.nan
lofar[opt_colname] = np.nan

c = ~np.isnan(lofar["lr_index_sel_2"])
indices = lofar["lr_index_sel_2"][c].astype(int)
lofar["category"][c] = opt_ir[indices]["category"]
lofar[ir_colname][c] = opt_ir[indices][ir_colname]
lofar[opt_colname][c] = opt_ir[indices][opt_colname]

In [ ]:
numbers_lofar_combined_bins = np.array([np.sum(lofar["category"] == c) 
                                        for c in range(len(numbers_combined_bins))])
numbers_lofar_combined_bins

## Skipped rerun iter section

In [ ]:
def plot_q_n_m(q_m, n_m):
    fig, a = plt.subplots()

    for i, q_m_k in enumerate(q_m):
        #plot(centers[i], q_m_old[i]/n_m_old[i])
        a = plt.subplot(4,4,i+1)
        if i not in [-1]:
            n_m_aux = n_m[i]/np.sum(n_m[i])
            lwidths = (n_m_aux/np.max(n_m_aux)*10).astype(float) + 1
            #print(lwidths)

            y_aux = q_m_k/n_m[i]
            
            y = y_aux
            mask = ~np.isnan(y_aux)
            y_aux = y_aux[mask]
            factor = np.max(y_aux[low:high])
            #print(y)
            x = centers[i]

            points = np.array([x, y]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)

            lc = LineCollection(segments, linewidths=lwidths, color=colors[i])

            a.add_collection(lc)

            #plot(centers[i], x/factor, color=colors[i-1])
            plt.xlim([12, 30])
            if i == 0:
                plt.xlim([10,23])
            plt.ylim([0, 1.2*factor])

    plt.subplots_adjust(left=0.125, 
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9,
                    wspace=0.4, 
                    hspace=0.2)
    return fig

In [ ]:
#create a copy of the radio catalogue for debugging
#original = lofar.copy()

In [ ]:
#lofar = original.copy()

In [ ]:
#Calculate the LR for radio sources using color
#returns the reliability and other parameters for each potential crossmatch to allow comparison
#the column lr_max is set to true for the crossmatch with the highest LR for each radio source

#the column lr_sel is set to true for the crossmatch with the highest LR for each radio source 
#but will later be set to false if the LR is below the threshold

def apply_ml_all(i, likelihood_ratio_function):
    
    #print(i)
    
    idx_0 = idx_i[idx_lofar == i]
    d2d_0 = d2d[idx_lofar == i]
    if len(idx_0) == 0:
        #print('no sources within search radius')
        temp = Table()
        col_index = Column(np.nan,name='lr_index_{}'.format(iteration))
        col_d2d = Column(np.nan,name='lr_dist_{}'.format(iteration))
        col_lr = Column(np.nan,name='lr_{}'.format(iteration))
        col_rel = Column(np.nan,name='lr_reliability_{}'.format(iteration))
        col_sourceid = Column(np.nan,name='Source_id')
        col_helpid = Column('',name='help_id')
        col_lr_max = Column(False,name='lr_max_{}'.format(iteration))
        col_lr_sel = Column(False,name='lr_sel_{}'.format(iteration))
        temp.add_columns([col_index,col_d2d,col_lr,col_rel,col_sourceid,col_helpid,col_lr_max,col_lr_sel])
        return temp,0
    
    category = catalogue["category"][idx_0].astype(int)
    mag = catalogue[opt_colname][idx_0]
    mag[category == 0] = catalogue[ir_colname][idx_0][category == 0]
    #print(mag)
    
    lofar_ra = lofar[i]["RA"]
    lofar_dec = lofar[i]["DEC"]
    lofar_pa = lofar[i]["PA"]
    lofar_maj_err = lofar[i]["E_Maj"]
    lofar_min_err = lofar[i]["E_Min"]
    c_ra = catalogue["ra"][idx_0]
    c_dec = catalogue["dec"][idx_0]
    c_ra_err = 1.0
    c_dec_err = 1.0
    
    sigma, sigma_maj, sigma_min = get_sigma_all(lofar_maj_err, lofar_min_err, lofar_pa, 
                      lofar_ra, lofar_dec, 
                      c_ra, c_dec, c_ra_err, c_dec_err)

    lr_0 = likelihood_ratio_function(mag, d2d_0.arcsec, sigma, category)
    #print(lr_0)

    chosen_index = np.argmax(lr_0)
    lr_sum = np.sum(lr_0)
    rel = lr_0/(lr_sum + (1-q0_total))
    helpid = catalogue['help_id'][idx_0]
    sourceid = (np.zeros(len(lr_0)) + np.unique(lofar['Source_id'])[i]).astype('int')
    col_index = Column(opt_ir_index[selection][idx_0],name='lr_index_{}'.format(iteration))
    col_d2d = Column(d2d_0.arcsec,name='lr_dist_{}'.format(iteration))
    col_lr = Column(lr_0,name='lr_{}'.format(iteration))
    col_rel = Column(rel,name='lr_reliability_{}'.format(iteration))
    col_sourceid = Column(sourceid,name='Source_id')
    col_helpid = Column(helpid,name='help_id')
    col_lr_max = Column(np.zeros(len(lr_0),dtype=bool),name='lr_max_{}'.format(iteration))
    col_lr_max[chosen_index] = True
    col_lr_sel = Column(np.zeros(len(lr_0),dtype=bool),name='lr_sel_{}'.format(iteration))
    col_lr_sel[chosen_index] = True

    #print(col_sourceid)
    #print(col_index,col_d2d,col_lr,col_rel,col_sourceid,col_helpid)
    
    temp = Table([col_index,col_d2d,col_lr,col_rel,col_sourceid,col_helpid,col_lr_max,col_lr_sel])

    lrs = np.max(col_lr)

    return (temp,lrs)

    #return (result)

In [ ]:
for j in range(10):
    iteration = j+3 
    print("Iteration {}".format(iteration))
    print("=============")
    ## Get new parameters
    # Number of matched sources per bin
    
    #select trusted crossmatches to recalculate Q0(c)
    if j==0:
        #for the first iteration need to use sources with LR>threshold as lr_sel hasn't been created for this iteration
        mask_sel = lofar['lr_2']>threshold
    else:
        mask_sel = lofar['lr_sel_{}'.format(iteration-1)]
    numbers_lofar_combined_bins = np.array([np.sum(lofar[mask_sel]["category"] == c) 
                                            for c in range(len(numbers_combined_bins))])
    print("numbers_lofar_combined_bins")
    print(numbers_lofar_combined_bins)
    #recompute Q0(c)
    Q_0_colour_est = numbers_lofar_combined_bins/len(np.unique(lofar['Source_id'])) ### Q_0
    Q_0_colour = q0_min_level(Q_0_colour_est, min_level=0.001)
    print("Q_0_colour")
    print(Q_0_colour)
    q0_total = np.sum(Q_0_colour)
    print("Q_0_total: ", q0_total)
    #Recompute q(m,c) for the new Q0(c)
    q_m = []
    # W1 only sources
    q_m.append(get_q_m(lofar[mask_sel][ir_colname][lofar[mask_sel]["category"] == 0], 
                   bin_list_ir, numbers_lofar_combined_bins[0], 
                   n_m[0], field.area, radius=radius))
    # Rest of the sources
    for i in range(1, len(numbers_lofar_combined_bins)):
        q_m.append(get_q_m(lofar[mask_sel][opt_colname][lofar[mask_sel]["category"] == i], 
                       bin_list_opt, numbers_lofar_combined_bins[i],
                       n_m[i], field.area, radius=radius))
    # q_m
    q_m_kde = []
    # W1 only sources
    q_m_kde.append(get_q_m_kde(lofar[mask_sel][ir_colname][lofar[mask_sel]["category"] == 0], 
                   centers[0], radius=radius, bandwidth=bandwidth_colour[0]))
    # Rest of the sources
    for i in range(1, len(numbers_lofar_combined_bins)):
        q_m_kde.append(get_q_m_kde(lofar[mask_sel][opt_colname][lofar[mask_sel]["category"] == i], 
                       centers[i], radius=radius, bandwidth=bandwidth_colour[i]))
    #set plot_intermediate to True if you want to see plots of n(m,c), q(m,c) and q(m,c)/n(m,c)
    plot_intermediate = False
    if plot_intermediate:
        fig = plt.figure(figsize=(15,15))
        for i, q_m_k in enumerate(q_m_kde):
            plt.subplot(5,5,i+1)
            plt.plot(centers[i], np.cumsum(q_m_k))
        #plt.savefig('{}/q0_{}.png'.format(idp, iteration))
        del fig
        fig = plt.figure(figsize=(15,15))
        for i, q_m_k in enumerate(q_m_kde):
            plt.subplot(5,5,i+1)
            plt.plot(centers[i], q_m_k/n_m[i])
        #plt.savefig('{}/q_over_n_{}.png'.format(idp, iteration))
        del fig
        fig = plot_q_n_m(q_m, n_m)
        #plt.savefig('{}/q_over_n_nice_{}.png'.format(idp, iteration))
        del fig
    ## Define new likelihood_ratio
    likelihood_ratio = MultiMLEstimator(Q_0_colour, n_m_kde, q_m_kde, centers)
    def ml(i):
        return apply_ml_all(i, likelihood_ratio)

    #create an empty table to store the crossmatch results with the same columns in it
    temp = Table()
    col_index = Column(name='lr_index_{}'.format(iteration))
    col_d2d = Column(name='lr_dist_{}'.format(iteration))
    col_lr = Column(name='lr_{}'.format(iteration))
    col_rel = Column(name='lr_reliability_{}'.format(iteration))
    col_sourceid = Column(name='Source_id')
    col_helpid = Column(name='help_id')
    temp.add_columns([col_index,col_d2d,col_lr,col_rel,col_sourceid,col_helpid])
    #run the LR calculation
    res = parallel_process(np.unique(idx_lofar), ml, n_jobs=n_cpus)
    test1 = np.transpose(res)
    test2 = test1[0].tolist()
    res = astropy.table.vstack(test2)
    #create a array of highest LR for each radio source
    lrs = test1[1]
    
    
    res["lr_{}".format(iteration)][np.isnan(res["lr_{}".format(iteration)])] = 0
    ## Get and apply the threshold calculated from Q0(c) 
    mask = res['lr_sel_{}'.format(iteration)]
    threshold = np.percentile(res[mask]['lr_{}'.format(iteration)], 100*(1 - q0_total))
    print("Threshold: ", threshold)
    
    #change lr_sel to False if the value is less than the threshold
    #for n,row in enumerate(res[res['lr_max_{}'.format(iteration)]]):
    #    if row['lr_{}'.format(iteration)]<threshold:
    #        mask = res['help_id']==row['help_id']
    #        res['lr_sel_{}'.format(iteration)][mask] = False             
    
    #Calculate the completeness and reliability as a function of the threshold
    #Use this to calculate the thershold that gives an equal completeness and reliability
    def completeness(lr, threshold, q0):
        n = len(lr)
        lrt = lr[lr < threshold]
        return 1. - np.sum((q0 * lrt)/(q0 * lrt + (1 - q0)))/(float(n)*q0)
    
    def reliability(lr, threshold, q0):
        n = len(lr)
        lrt = lr[lr > threshold]
        return 1. - np.sum((1. - q0)/(q0 * lrt + (1 - q0)))/(float(n)*q0)
    completeness_v = np.vectorize(completeness, excluded=[0])
    reliability_v = np.vectorize(reliability, excluded=[0])
    thresholds = np.arange(0., 10., 0.001)
    mask = lofar['lr_max_{}'.format(iteration)]
    lr_temp = lofar['lr_{}'.format(iteration)][mask]
    completeness_t = completeness_v(lr_temp , thresholds, Q0)
    reliability_t = reliability_v(lr_temp , thresholds, Q0)
    diff = abs(completeness_t-reliability_t)
    mask = ~np.isnan(diff)
    index = np.argmin(diff[mask])
    threshold = thresholds[mask][index]
    plt.plot(thresholds,completeness_t)
    plt.plot(thresholds,reliability_t)
    plt.vlines(threshold,0,1)
    plt.ylim(0.9,1.0)
    plt.show()
    print("updated Threshold: ", threshold)

    for n,row in enumerate(res[res['lr_max_{}'.format(iteration)]]):#####
        if row['lr_{}'.format(iteration)]<threshold:
            mask = res['help_id']==row['help_id']
            res['lr_sel_{}'.format(iteration)][mask] = False
        #only include if you uncomment the earlier redoing of lr_sel after calculating the first threshold
        #based on Q0(c)
        #else:
        #    mask = res['help_id']==row['help_id']
        #    res['lr_sel_{}'.format(iteration)][mask] = True
    
    #join the crossmatch data to te radio table the first time this is done an outer join
    #is needed as there are multiple crossmatches for each radio source and all of these are needed
    #For subsequent iterations a hstack can be used 
    if j==0:
        lofar = join(lofar,res,keys='Source_id',join_type='outer')
    else:
        res.remove_column('Source_id')
        lofar = astropy.table.hstack([lofar,res])
    
    if plot_intermediate:
        fig = plt.figure(figsize=(15,6))
        plt.subplot(1,2,1)
        plt.hist(lofar[lofar["lr_{}".format(iteration)] != 0]["lr_{}".format(iteration)], bins=200)
        plt.vlines([threshold], 0, 1000)
        plt.ylim([0,1000])
        plt.subplot(1,2,2)
        plt.hist(np.log10(lofar[lofar["lr_{}".format(iteration)] != 0]["lr_{}".format(iteration)]+1), bins=200)
        plt.vlines(np.log10(threshold+1), 0, 1000)
        ticks, _ = plt.xticks()
        plt.xticks(ticks, ["{:.1f}".format(10**t-1) for t in ticks])
        plt.ylim([0,1000])
        #plt.savefig('{}/lr_distribution_{}.png'.format(idp, iteration))
        del fig
    ## Apply the threshold

    ## Enter changes into the catalogue
    # Clear aux columns
    lofar["category"] = np.nan
    lofar[ir_colname] = np.nan
    lofar[opt_colname] = np.nan
    # Update data
    c = ~np.isnan(lofar["lr_index_{}".format(iteration)])
    indices = lofar["lr_index_{}".format(iteration)][c].astype(int)
    lofar["category"][c] = opt_ir[indices]["category"]
    lofar[ir_colname][c] = opt_ir[indices][ir_colname]
    lofar[opt_colname][c] = opt_ir[indices][opt_colname]
    '''# Save the data
    if save_intermediate:
        lofar[mask_sel].write("{}/lofar[mask_sel]_m{}.fits".format(idp, iteration), format="fits")'''
    ## Compute number of changes
    n_changes = np.sum((
            lofar["lr_index_{}".format(iteration)] != lofar["lr_index_{}".format(iteration-1)]) & 
            ~np.isnan(lofar["lr_index_{}".format(iteration)]) &
            ~np.isnan(lofar["lr_index_{}".format(iteration-1)]))
    print("N changes: ", n_changes)
    t_changes = np.sum((
            lofar["lr_index_{}".format(iteration)] != lofar["lr_index_{}".format(iteration-1)]))
    print("T changes: ", t_changes)
    ## Check changes
    plt.show()
    if n_changes == 0:
        break
    else:
        print("******** continue **********")

In [ ]:
Table.write(lofar,'data/xmatch_all_20181113',format='fits')

In [ ]:
lofar